# Практика с лекции

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [2]:
!curl https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv -O titanic.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 22 60302   22 13710    0     0  30515      0  0:00:01 --:--:--  0:00:01 30534
100 60302  100 60302    0     0   120k      0 --:--:-- --:--:-- --:--:--  120k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: titanic.csv


In [3]:
df = spark.read.csv('titanic.csv', inferSchema=True, header=True)

In [4]:
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [5]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [6]:
df_filtered = df.select('Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked')

In [7]:
df_filtered.show(5)

+--------+------+------+----+-------+--------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|
+--------+------+------+----+-------+--------+
|       0|     3|  male|22.0|   7.25|       S|
|       1|     1|female|38.0|71.2833|       C|
|       1|     3|female|26.0|  7.925|       S|
|       1|     1|female|35.0|   53.1|       S|
|       0|     3|  male|35.0|   8.05|       S|
+--------+------+------+----+-------+--------+
only showing top 5 rows



In [8]:
df_filtered.groupBy('Embarked').count().show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   77|
|    null|    2|
|       C|  168|
|       S|  644|
+--------+-----+



In [9]:
df_filtered.describe().show()

+-------+-------------------+------------------+------+------------------+-----------------+--------+
|summary|           Survived|            Pclass|   Sex|               Age|             Fare|Embarked|
+-------+-------------------+------------------+------+------------------+-----------------+--------+
|  count|                891|               891|   891|               714|              891|     889|
|   mean| 0.3838383838383838| 2.308641975308642|  null| 29.69911764705882| 32.2042079685746|    null|
| stddev|0.48659245426485753|0.8360712409770491|  null|14.526497332334035|49.69342859718089|    null|
|    min|                  0|                 1|female|              0.42|              0.0|       C|
|    max|                  1|                 3|  male|              80.0|         512.3292|       S|
+-------+-------------------+------------------+------+------------------+-----------------+--------+



In [10]:
df_filtered = df_filtered.na.fill({'Age':30,'Embarked':'S'})

In [11]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

In [12]:
indexer = StringIndexer(inputCol='Sex', outputCol='SexInd')
indexerTrained = indexer.fit(df_filtered)
df_upd = indexerTrained.transform(df_filtered)

indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkedInd')
indexerTrained = indexer.fit(df_upd)
df_upd = indexerTrained.transform(df_upd)

In [13]:
onehot = OneHotEncoder(inputCol='EmbarkedInd', outputCol='EmbarkedOhe')
onehotTrained = onehot.fit(df_upd)
df_upd = onehotTrained.transform(df_upd)

In [14]:
df_upd.show(5)

+--------+------+------+----+-------+--------+------+-----------+-------------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|SexInd|EmbarkedInd|  EmbarkedOhe|
+--------+------+------+----+-------+--------+------+-----------+-------------+
|       0|     3|  male|22.0|   7.25|       S|   0.0|        0.0|(2,[0],[1.0])|
|       1|     1|female|38.0|71.2833|       C|   1.0|        1.0|(2,[1],[1.0])|
|       1|     3|female|26.0|  7.925|       S|   1.0|        0.0|(2,[0],[1.0])|
|       1|     1|female|35.0|   53.1|       S|   1.0|        0.0|(2,[0],[1.0])|
|       0|     3|  male|35.0|   8.05|       S|   0.0|        0.0|(2,[0],[1.0])|
+--------+------+------+----+-------+--------+------+-----------+-------------+
only showing top 5 rows



In [15]:
from pyspark.ml.feature import VectorAssembler

In [16]:
assembler = VectorAssembler(inputCols=['Pclass','Age','Fare','SexInd','EmbarkedOhe'], outputCol='Features')

In [17]:
df_upd = assembler.transform(df_upd)
df_upd.show(5)

+--------+------+------+----+-------+--------+------+-----------+-------------+--------------------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|SexInd|EmbarkedInd|  EmbarkedOhe|            Features|
+--------+------+------+----+-------+--------+------+-----------+-------------+--------------------+
|       0|     3|  male|22.0|   7.25|       S|   0.0|        0.0|(2,[0],[1.0])|[3.0,22.0,7.25,0....|
|       1|     1|female|38.0|71.2833|       C|   1.0|        1.0|(2,[1],[1.0])|[1.0,38.0,71.2833...|
|       1|     3|female|26.0|  7.925|       S|   1.0|        0.0|(2,[0],[1.0])|[3.0,26.0,7.925,1...|
|       1|     1|female|35.0|   53.1|       S|   1.0|        0.0|(2,[0],[1.0])|[1.0,35.0,53.1,1....|
|       0|     3|  male|35.0|   8.05|       S|   0.0|        0.0|(2,[0],[1.0])|[3.0,35.0,8.05,0....|
+--------+------+------+----+-------+--------+------+-----------+-------------+--------------------+
only showing top 5 rows



# Домашнее задание

### Взять данные.

In [18]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').getOrCreate()
sc = spark.sparkContext

### Загрузить в PySpark.

In [19]:
df = spark.read.csv('iris.CSV', header=True, inferSchema=True)
df.show(5)

+------------+-----------+------------+-----------+-------+-----------+
|sepal_length|sepal_width|petal_length|petal_width|variety|variety_num|
+------------+-----------+------------+-----------+-------+-----------+
|         5.1|        3.5|         1.4|        0.2| Setosa|          0|
|         4.9|        3.0|         1.4|        0.2| Setosa|          0|
|         4.7|        3.2|         1.3|        0.2| Setosa|          0|
|         4.6|        3.1|         1.5|        0.2| Setosa|          0|
|         5.0|        3.6|         1.4|        0.2| Setosa|          0|
+------------+-----------+------------+-----------+-------+-----------+
only showing top 5 rows



In [20]:
df.describe().show()

+-------+------------------+-------------------+------------------+------------------+---------+------------------+
|summary|      sepal_length|        sepal_width|      petal_length|       petal_width|  variety|       variety_num|
+-------+------------------+-------------------+------------------+------------------+---------+------------------+
|  count|               150|                150|               150|               150|      150|               150|
|   mean| 5.843333333333335|  3.057333333333334|3.7580000000000027| 1.199333333333334|     null|               1.0|
| stddev|0.8280661279778637|0.43586628493669793|1.7652982332594662|0.7622376689603467|     null|0.8192319205190406|
|    min|               4.3|                2.0|               1.0|               0.1|   Setosa|                 0|
|    max|               7.9|                4.4|               6.9|               2.5|Virginica|                 2|
+-------+------------------+-------------------+------------------+-----

### При помощи VectorAssembler преобразовать все колонки с признаками в одну (использовать Pipeline — опционально).

In [21]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler

pl = Pipeline(stages=[VectorAssembler(inputCols=['sepal_length','sepal_width','petal_length','petal_width'], outputCol='Features')])
plTrained = pl.fit(df)
df_transf = plTrained.transform(df)
df_transf.show(5)

+------------+-----------+------------+-----------+-------+-----------+-----------------+
|sepal_length|sepal_width|petal_length|petal_width|variety|variety_num|         Features|
+------------+-----------+------------+-----------+-------+-----------+-----------------+
|         5.1|        3.5|         1.4|        0.2| Setosa|          0|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2| Setosa|          0|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2| Setosa|          0|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2| Setosa|          0|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2| Setosa|          0|[5.0,3.6,1.4,0.2]|
+------------+-----------+------------+-----------+-------+-----------+-----------------+
only showing top 5 rows



### Разбить данные на train и test.

In [22]:
train, test = df_transf.randomSplit([0.8,0.2], seed=42)

### Создать модель логистической регреcсии или модель дерева и обучить её.
Воспользоваться MulticlassClassificationEvaluator для оценки качества на train и test множестве.

In [23]:
# Решил сделать обе
from pyspark.ml.classification import LogisticRegression

In [24]:
lr = LogisticRegression(featuresCol='Features', labelCol='variety_num')
lrTrained = lr.fit(train)
df_train = lrTrained.transform(train)
df_test = lrTrained.transform(test)
df_test.show(5)

+------------+-----------+------------+-----------+-------+-----------+-----------------+--------------------+--------------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|variety|variety_num|         Features|       rawPrediction|         probability|prediction|
+------------+-----------+------------+-----------+-------+-----------+-----------------+--------------------+--------------------+----------+
|         4.4|        3.0|         1.3|        0.2| Setosa|          0|[4.4,3.0,1.3,0.2]|[54.8316731860621...|[1.0,5.8045981223...|       0.0|
|         4.6|        3.2|         1.4|        0.2| Setosa|          0|[4.6,3.2,1.4,0.2]|[59.0189684197576...|[1.0,1.7180006172...|       0.0|
|         4.6|        3.6|         1.0|        0.2| Setosa|          0|[4.6,3.6,1.0,0.2]|[76.5905029625326...|[1.0,1.3749820140...|       0.0|
|         4.8|        3.1|         1.6|        0.2| Setosa|          0|[4.8,3.1,1.6,0.2]|[52.2432095666216...|[1.0,1.6438217837...|       0.0|

In [25]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

ev = MulticlassClassificationEvaluator(labelCol='variety_num')
ev.evaluate(df_test)

1.0

In [26]:
ev.evaluate(df_train)

0.9841269841269842

Теперь Decision Tree

In [28]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(featuresCol='Features', labelCol='variety_num')
dtTrained = dt.fit(train)
df_train = dtTrained.transform(train)
df_test = dtTrained.transform(test)
df_test.show(5)

+------------+-----------+------------+-----------+-------+-----------+-----------------+--------------+-------------+----------+
|sepal_length|sepal_width|petal_length|petal_width|variety|variety_num|         Features| rawPrediction|  probability|prediction|
+------------+-----------+------------+-----------+-------+-----------+-----------------+--------------+-------------+----------+
|         4.4|        3.0|         1.3|        0.2| Setosa|          0|[4.4,3.0,1.3,0.2]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.6|        3.2|         1.4|        0.2| Setosa|          0|[4.6,3.2,1.4,0.2]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.6|        3.6|         1.0|        0.2| Setosa|          0|[4.6,3.6,1.0,0.2]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.8|        3.1|         1.6|        0.2| Setosa|          0|[4.8,3.1,1.6,0.2]|[39.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|         4.9|        3.1|         1.5|        0.1| Setosa|          0|[4.9,3.1,1.5,0.1]|[

In [29]:
ev.evaluate(df_test)

1.0

In [30]:
ev.evaluate(df_train)

0.9920613938459278